In [ ]:
from sys import path
from os import getcwd, environ, walk

path.insert(0, getcwd())
path.insert(0, getcwd() + "/modules/")
environ["CUDA_VISIBLE_DEVICES"] = "3"

from modules.DataMod import DataSet

import matplotlib.pyplot as plt
import random as rd

from keras import models 

In [ ]:
tinyDataSet, cifarDataSet, cifarAndTinyDataSet = DataSet(), DataSet(), DataSet()

tinyDataSet = tinyDataSet.load_rafael_tinyImagenet_64x64_noise_data()
cifarDataSet = cifarDataSet.load_rafael_cifar_10_noise_data()

# concatenates the datasets
cifarAndTinyDataSet = cifarAndTinyDataSet.concatenateDataSets(cifarDataSet, tinyDataSet)

In [ ]:
def plot_model_graphic(model, dataset, output_path):
        # plots the model
        
        print(dataset.x_test)
        print(dataset.x_test[0].shape)

        plt.figure(figsize=(10, 10))

        columns = 3
        rows = 5

        for idx in [rd.randint(0, len(dataset.x_test)) for i in range(rows)]:
                plt.subplot(rows, columns, columns*idx + 1)
                plt.title("Original Image")
                plt.imshow(dataset.x_test[idx])
                plt.subplot(rows, columns, columns*idx + 2)
                plt.title("Goal Image")
                plt.imshow(dataset.y_test[idx])
                plt.subplot(rows, columns, columns*idx + 3)
                plt.title("Predicted Image")
                plt.imshow(model.predict(dataset.x_test[idx].reshape(1, 64, 64, 3))[0])

        plt.savefig(output_path)
        plt.close()

In [ ]:
NNmodels = {}

for path in ["AutoEncoder-2.3-64x64.json", "ResidualAutoEncoder-0.1-64x64.json", "Unet2.3-64x64.json"]:
        # reads the model
        with open("models/arch/"+path, "r") as json_file:
                model = models.model_from_json(json_file.read())
                NNmodels[model.name] = model


for model in NNmodels:
    for (dirpath, dirnames, filenames) in walk("logs/run1/weights/"):
        for filename in filenames:
            if filename.startswith(model):

                NNmodels[model].load_weights(filename)

                for dataset in [cifarAndTinyDataSet, cifarDataSet, tinyDataSet]:
                    if dataset.name in filename:
                        print("Evaluating model: " + filename.split(".")[0])
                        loss, ssim, tssim, psnrb = NNmodels[model].evaluate(x = dataset.x_test, y = dataset.y_test, verbose = 2)
                        print("Loss: " + str(loss))
                        print("SSIM: " + str(ssim))
                        print("3SSIM: " + str(tssim))
                        print("PSNRB: " + str(psnrb))
                        print("\n")

                        plot_model_graphic(NNmodels[model], dataset, "/logs/run1/plots/"+filename.split(".")[0]+".png")